In [30]:
import pandas
import requests
import io
import zipfile

In [31]:
def get_zipped_csv_from_url(url : str, csv_name : str, num_rows : int) -> pandas.DataFrame:
    web_file = requests.get(url=url)
    zip = zipfile.ZipFile(io.BytesIO(web_file.content))
    #data = pandas.read_csv(zip.open(csv_name), nrows=num_rows,usecols=['Date received','Issue'])
    data = pandas.read_csv(zip.open(csv_name), nrows=num_rows)
    return pandas.DataFrame(data)

    #print(df)
df = get_zipped_csv_from_url("https://files.consumerfinance.gov/ccdb/complaints.csv.zip","complaints.csv",10)

In [48]:
id = "1228393702244134912"
endpoint = "https://api.twitter.com/2/tweets/"+id
credential_path = "/Users/at/API Credentials/TwitterCreds.txt"
file = open(credential_path, "r")
#read first line for bearer
token = file.readline()
bearer_token = "Bearer " + token
headers = {"Authorization" : bearer_token}
response = requests.get(endpoint,headers=headers).json()
print(response)


{'data': {'id': '1228393702244134912', 'text': 'What did the developer write in their Valentine’s card?\n  \nwhile(true) {\n    I = Love(You);  \n}'}}
